In [20]:
import pickle
import json
import numpy as np
import pandas as pd

# Load trained model
with open("heart_disease_model.pkl", "rb") as file:
    model = pickle.load(file)

# Load heart disease types from JSON
with open("heart_disease_info.json", "r") as file:
    heart_disease_info_weighted = json.load(file)

# Function to predict heart disease
def predict_heart_disease(age, gender, chol, bp, hr, diabetes, symptoms):
    # Convert categorical values to match training data
    gender_male = 1 if gender.lower() == "male" else 0
    diabetes_yes = 1 if diabetes.lower() == "yes" else 0
    
    # Create DataFrame with correct column names
    input_data = pd.DataFrame([[age, chol, bp, hr, gender_male, diabetes_yes]], 
                              columns=["Age", "Cholesterol", "Blood Pressure", "Heart Rate", "Gender_Male", "Diabetes_Yes"])
    
    # Predict heart disease (0 = No, 1 = Yes)
    disease_present = model.predict(input_data)[0]

    if disease_present == 0:
        return "No heart disease detected. Stay healthy! 😊"
    
    # Identify the type of heart disease using symptoms
    return identify_heart_disease_type(symptoms)

# Function to determine heart disease type based on symptoms
def identify_heart_disease_type(symptoms):
    symptom_scores = {}

    for disease, details in heart_disease_info_weighted.items():
        total_score = 0
        for symptom in symptoms:
            symptom = symptom.strip().lower()
            for key, value in details["Symptoms"].items():
                if symptom in key.lower():
                    total_score += value
        symptom_scores[disease] = total_score

    # Find the disease with the highest matching symptoms
    best_match = max(symptom_scores, key=symptom_scores.get)

    return f"Detected: {best_match}\nPossible Causes: {', '.join(heart_disease_info_weighted[best_match]['Cause'])}\nSuggested Treatment: {', '.join(heart_disease_info_weighted[best_match]['Treatment'])}"

# User input
age = int(input("Enter age: "))
gender = input("Enter gender (Male/Female): ")
chol = int(input("Enter cholesterol level: "))
bp = int(input("Enter blood pressure: "))
hr = int(input("Enter heart rate: "))
diabetes = input("Do you have diabetes? (Yes/No): ")
symptoms = input("Enter symptoms separated by commas: ").split(",")

# Predict and display result
result = predict_heart_disease(age, gender, chol, bp, hr, diabetes, symptoms)
print("\n🔹 Result:")
print(result)


Enter age:  75
Enter gender (Male/Female):  male
Enter cholesterol level:  228
Enter blood pressure:  119
Enter heart rate:  66
Do you have diabetes? (Yes/No):  no
Enter symptoms separated by commas:  chest pain, fatigue,Cold sweats



🔹 Result:
Detected: Myocardial Infarction (Heart Attack)
Possible Causes: Atherosclerosis (plaque buildup in arteries), Blood clot blocking coronary artery
Suggested Treatment: Thrombolytics (to dissolve the clot), Aspirin (to prevent further clotting), Beta-blockers (to reduce heart workload), Angioplasty (to restore blood flow), Bypass surgery (to bypass blocked arteries)


In [2]:
print("Model Type:", type(model))


Model Type: <class 'numpy.ndarray'>
